In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate();

 Activating environment at `C:\Users\ALP\Desktop\Sioux\ForneyInternship\ForneyLab.jl\Project.toml`


In [2]:
using ForneyLab
using LinearAlgebra,Flux
using Random
import Distributions: pdf, MvNormal, rand

Random.seed!(1) # Set random seed

n = 10 # Number of datapoints

# Data
true_x = [0.5, 0.5] # True value for latent variable x
A = diageye(2) # diageye returns an identity matrix of type Diagonal
b = zeros(2)
Sigma_y = 0.1*[2 1; 1 1] # Observation covariance

y_hat_mat = rand(MvNormal(A*true_x + b, Sigma_y), n)' # Data matrix
y_hat = [y_hat_mat[i,:] for i=1:n] # Unfold data matrix in a vector of vectors
;

┌ Warning: This version of CUDA.jl only supports NVIDIA drivers for CUDA 10.1 or higher (yours is for CUDA 9.2.0)
└ @ CUDA C:\Users\ALP\.julia\packages\CUDA\mbPFj\src\initialization.jl:100


In [6]:
f(x)=x
num_samples = 1000
g = FactorGraph() # Initiate a new factor graph
# Model
@RV x ~ GaussianMeanVariance(
          placeholder(:mu_x, dims=(2,)), 
          placeholder(:Sigma_x, dims=(2,2))) # Prior with placeholders for statistics

@RV x_bar ~ Cvi(x,g=f,opt=Descent(1e-5),num_samples=num_samples,num_iterations=Int64(100000))
@RV y ~ GaussianMeanVariance(A*x_bar + b, Sigma_y) # Likelihood

placeholder(y, :y, dims=(2,)) # Indicate observed variable; dims indicates the dimensionality of the data
algo = messagePassingAlgorithm(x) # Generate algorithm
source_code = algorithmSourceCode(algo)
eval(Meta.parse(source_code)) # Load algorithm

#println(source_code) # Inspect the algorithm
# Prior statistics
mu_x_0 = zeros(2)
Sigma_x_0 = 0.1*diageye(2)

# Perform inference
mu_x = deepcopy(mu_x_0)
Sigma_x = deepcopy(Sigma_x_0)
for i = 1:10
    data = Dict(:mu_x => mu_x,
                :Sigma_x => Sigma_x,
                :y => y_hat[i]) # Prepare dictionary with prior statistics and present datapoint
    
    marginals = step!(data) # Execute the algorithm
    
    # Today's posterior is tomorrow's prior
    mu_x = mean(marginals[:x])
    Sigma_x = cov(marginals[:x])
    println("μ=$(mu_x)")
end


μ=[0.0729376513458748, 0.20758410336607036]
μ=[0.09312788577090446, 0.24251153199866227]
μ=[0.09002182989469626, 0.2739013105782836]
μ=[0.2941930730540115, 0.2578236409894248]
μ=[0.3369883722479696, 0.31702971355520315]
μ=[0.3725269017373934, 0.37935040581026525]
μ=[0.4042965793129016, 0.38590029349025456]
μ=[0.42688276728458996, 0.40244919541759905]
μ=[0.4126485176597683, 0.36166762451735524]
μ=[0.42355908579791823, 0.3597804219372661]


In [4]:
# f(x)=x
# num_samples = 1000
# g = FactorGraph() # Initiate a new factor graph
# # Model
# @RV x ~ GaussianMeanVariance(
#           placeholder(:mu_x, dims=(2,)), 
#           placeholder(:Sigma_x, dims=(2,2))) # Prior with placeholders for statistics

# @RV x_bar ~ Cvi(x,g=f,opt=Descent(1e-4),num_samples=num_samples,num_iterations=Int64(100000),convergence_optimizer = ConvergenceParamsFE())
# @RV y ~ GaussianMeanVariance(A*x_bar + b, Sigma_y) # Likelihood

# placeholder(y, :y, dims=(2,)) # Indicate observed variable; dims indicates the dimensionality of the data
# algo = messagePassingAlgorithm(x) # Generate algorithm
# source_code = algorithmSourceCode(algo)
# eval(Meta.parse(source_code)) # Load algorithm

# #println(source_code) # Inspect the algorithm
# # Prior statistics
# mu_x_0 = zeros(2)
# Sigma_x_0 = 0.1*diageye(2)

# # Perform inference
# mu_x = deepcopy(mu_x_0)
# Sigma_x = deepcopy(Sigma_x_0)
# for i = 1:10
#     data = Dict(:mu_x => mu_x,
#                 :Sigma_x => Sigma_x,
#                 :y => y_hat[i]) # Prepare dictionary with prior statistics and present datapoint
    
#     marginals = step!(data) # Execute the algorithm
    
#     # Today's posterior is tomorrow's prior
#     mu_x = mean(marginals[:x])
#     Sigma_x = cov(marginals[:x])
#     println("μ=$(mu_x)")
# end


In [5]:
f(x)=x
num_samples = 1000
g = FactorGraph() # Initiate a new factor graph
# Model
@RV x ~ GaussianMeanVariance(
          placeholder(:mu_x, dims=(2,)), 
          placeholder(:Sigma_x, dims=(2,2))) # Prior with placeholders for statistics

@RV x_bar ~ Cvi(x,g=f,opt=Descent(1e-4),num_samples=num_samples,num_iterations=Int64(100000),convergence_optimizer = ConvergenceParamsMC())
@RV y ~ GaussianMeanVariance(A*x_bar + b, Sigma_y) # Likelihood

placeholder(y, :y, dims=(2,)) # Indicate observed variable; dims indicates the dimensionality of the data
algo = messagePassingAlgorithm(x) # Generate algorithm
source_code = algorithmSourceCode(algo)
eval(Meta.parse(source_code)) # Load algorithm

#println(source_code) # Inspect the algorithm
# Prior statistics
mu_x_0 = zeros(2)
Sigma_x_0 = 0.1*diageye(2)

# Perform inference
mu_x = deepcopy(mu_x_0)
Sigma_x = deepcopy(Sigma_x_0)
for i = 1:10
    data = Dict(:mu_x => mu_x,
                :Sigma_x => Sigma_x,
                :y => y_hat[i]) # Prepare dictionary with prior statistics and present datapoint
    
    marginals = step!(data) # Execute the algorithm
    
    # Today's posterior is tomorrow's prior
    mu_x = mean(marginals[:x])
    Sigma_x = cov(marginals[:x])
    println("μ=$(mu_x)")
end


μ=[0.12286397721450719, 0.2645076945955985]
μ=[0.14377145148828413, 0.29082332629401736]
μ=[0.13476387189919517, 0.31578904114559103]
μ=[0.36817593091113593, 0.30720203719277517]
μ=[0.4122225884434894, 0.36781263700490807]
μ=[0.44792256708656397, 0.431040733254048]
μ=[0.4761327438695072, 0.4343437301014103]
μ=[0.4954062225800522, 0.44836816787447686]
μ=[0.4719424425503612, 0.40070911091663086]
μ=[0.47915810333269826, 0.3961710179540145]
